In [5]:
from elasticsearch import Elasticsearch
import requests
from tqdm.auto import tqdm

In [6]:
from mistralai import Mistral
from mistralai.models import UserMessage
import os
from dotenv import load_dotenv
import httpx
# from mistral_common.tokens.tokenizers.mistral import MistralTokenizer
import tiktoken


In [9]:
es_client = Elasticsearch('http://localhost:9200')
print(es_client.info())


{'name': 'abff37edfb02', 'cluster_name': 'docker-cluster', 'cluster_uuid': '1IsKjrBrSRunUPmdNvkJUw', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


In [10]:
# get the data from json File

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url, verify=False)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

/home/codespace/.local/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'github.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/codespace/.local/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'github.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/codespace/.local/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'raw.githubusercontent.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [11]:
# No. of the docs 
len(documents)

948

In [12]:
# Print first 5 docs 
for index, value in enumerate(documents[:5]):
    print(f"Index: {index}, Value: {value}")
    print('---------------------------------')
    

Index: 0, Value: {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.", 'section': 'General course-related questions', 'question': 'Course - When will the course start?', 'course': 'data-engineering-zoomcamp'}
---------------------------------
Index: 1, Value: {'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites', 'section': 'General course-related questions', 'question': 'Course - What are the prerequisites for this course?', 'course': 'data-engineering-zoomcamp'}
---------------------------------
Index: 2, Value: {'text': "Yes, even if you don't regi

In [13]:
# if es_client.indices.exists(index=index_name):
#     es_client.indices.delete(index=index_name)
#     print(f"Deleted existing index: {index_name}")


In [14]:
# Index doc by 'course' value and the rest is the text 

index_settings = {
    "settings":{
        "number_of_shards":1,
        "number_of_replicas":0
    },
    "mappings":{
        "properties":{
            "text":{"type":"text"},
            "section":{"type":"text"},
            "question":{"type":"text"},
            "course":{"type":"keyword"},
        }
    }
}

index_name = "faq"
es_client.indices.create(index=index_name, body=index_settings)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'faq'})

In [15]:
# Set the index for the whole docs

for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [16]:
def elastic_search(query, size, course_filter=None):

    search_query = {
        "size": size,
        "query": {
            "bool": {
                "must": [
                    {
                        "multi_match": {
                            "query": query,
                            "fields": ["question^4", "text"],
                            "type": "best_fields"
                        }
                    }
                ]
            }
        }
    }

    # Add filter 
    if course_filter:
        search_query["query"]["bool"]["filter"] = {
            "term": {
                "course": course_filter
            }
        }


    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []

    for hit in response['hits']['hits']:
        result_docs.append(hit)     
    
    return result_docs

In [17]:
# Searching 
f_query = "How do execute a command on a Kubernetes pod?"
elastic_search(query=f_query, size=5, course_filter=None)

[{'_index': 'faq',
  '_id': 'cihUZpcB9JtM4P-nUJtU',
  '_score': 44.50556,
  '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
   'section': '5. Deploying Machine Learning Models',
   'question': 'How do I debug a docker container?',
   'course': 'machine-learning-zoomcamp'}},
 {'_index': 'faq',
  '_id': 'AShUZpcB9JtM4P-nUpwI',
  '_score': 35.433445,
  '_source': {'text': 'Deploy and Access the Kubernetes Dashboard\nLuke',
   'section': '10. Kubernetes and TensorFlow Serving',
   'question': 'Kubernetes-dashboard',
   'course': 'machine-learning-zoomcamp'}},
 {'_index': 'faq',
  '_id': 'kihUZpcB9JtM4P-nUJuz',
  '_score': 33.70974,
  '_source': {'text': 'You can copy files from your local machine 

In [18]:
# Filter
sec_query = "How do copy a file to a Docker container?"
elastic_search(query=sec_query, size=3, course_filter="machine-learning-zoomcamp")

[{'_index': 'faq',
  '_id': 'cihUZpcB9JtM4P-nUJtU',
  '_score': 73.38676,
  '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
   'section': '5. Deploying Machine Learning Models',
   'question': 'How do I debug a docker container?',
   'course': 'machine-learning-zoomcamp'}},
 {'_index': 'faq',
  '_id': 'kShUZpcB9JtM4P-nUJuw',
  '_score': 66.688705,
  '_source': {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:

In [19]:
# loads variables from .env
load_dotenv()  

True

In [20]:
api_key = os.getenv("API_KEY")

In [21]:
client = Mistral(api_key = api_key, client=httpx.Client(verify=False))

In [22]:
def build_prompt(query, search_results):
    context_template = "Q: {question}\nA: {text}"
    
    context_parts = []
    for doc in search_results:
        source = doc['_source']
        context_parts.append(context_template.format(question=source.get('question', ''), text=source.get('text', '')))
    
    context = "\n\n".join(context_parts)

    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

    prompt = prompt_template.format(question=query, context=context)
    print('The length of the resulting prompt:', len(prompt))

    encoding = tiktoken.encoding_for_model("gpt-4o")
    encoded_prompt = encoding.encode(prompt)
    num_tokens = len(encoded_prompt)
    
    ## Number of tokens
    print("The number of token",num_tokens)

    return prompt
    

In [23]:
def llm(prompt):
    response = client.chat.complete(
        model= "mistral-medium-latest",
        messages=[UserMessage(content=prompt)],
    )


    return response.choices[0].message.content

In [24]:
def rag(query):
    search_results = elastic_search(query=sec_query, size=3, course_filter="machine-learning-zoomcamp")
    # Initialize tokenizer
    prompt = build_prompt(sec_query, search_results)
    answer = llm(prompt)
    return answer

In [25]:
answer = rag(sec_query)
print("\n Answer:", answer)

The length of the resulting prompt: 1446
The number of token 320

 Answer: To copy a file to a Docker container, you can use the `docker cp` command. The syntax is as follows:

```bash
docker cp /path/to/local/file_or_directory container_id:/path/in/container
```

This command allows you to copy files or directories from your local machine into a running Docker container.

*(Source: Hrithik Kumar Advani, Gopakumar Gopinathan)*
